In [4]:
import pandas as pd

# 1. 加载数据
# 假设基因表达数据已经是CSV文件，并且表型数据也是CSV格式
gene_expression_file = r"C:\Users\Lyttl\Desktop\rice\shuidao\data\expression_rpkm_new_test2_T.csv"
phenotype_file = r"C:\Users\Lyttl\Desktop\rice\shuidao\data\phenos_Entry_Treatment_Replicate_sorted_anaylize.csv"

# 加载数据
gene_df = pd.read_csv(gene_expression_file)
phenotype_df = pd.read_csv(phenotype_file)

# 2. 数据清理和格式化
# 检查表型数据中的列名
# 检查 gene_df 和 phenotype_df 中的列名
print("Gene expression columns:", gene_df.columns)
print("Phenotype columns:", phenotype_df.columns)

# 确保表型数据包含 'Entry_Treatment_Replicate' 和 'CCI' 列
phenotype_df = phenotype_df[['Entry_Treatment_Replicate', 'Fecundity']]

# 3. 提取目标变量（CCI）
target = phenotype_df['Fecundity']

# 4. 合并数据
# 将基因表达数据与表型数据按 'Entry_Treatment_Replicate' 进行合并
combined_df = pd.merge(gene_df, phenotype_df, on='Entry_Treatment_Replicate', how='inner')

# 5. 预处理数据
# 如果需要，可以对数据进行标准化处理（例如使用StandardScaler）
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# 假设gene_df已经是数值特征，排除 'Entry_Treatment_Replicate' 和 'CCI' 列
features = combined_df.drop(columns=['Entry_Treatment_Replicate', 'Fecundity'])
features_scaled = scaler.fit_transform(features)

# 7. 将合并的数据输出为CSV文件
combined_df.to_csv(r"C:\Users\Lyttl\Desktop\rice\shuidao\data\combined_gene_phenotype_data_Fecundity.csv", index=False, header=True)

# 输出成功信息
print("Merged data has been saved as 'combined_gene_phenotype_data_Fecundity.csv'.")


Gene expression columns: Index(['Entry_Treatment_Replicate', 'OS01T0100700-01', 'OS01T0100900-01',
       'OS01T0101700-00', 'OS01T0102300-01', 'OS01T0102600-01',
       'OS01T0102700-01', 'OS01T0102900-01', 'OS01T0105900-01',
       'OS01T0106800-01',
       ...
       'OS09T0558100-01', 'OS09T0559600-01', 'OS09T0560300-01',
       'OS09T0560900-01', 'OS09T0563200-01', 'OS09T0565200-02',
       'OS09T0567300-01', 'OS09T0568800-01', 'OS09T0568900-01',
       'OS09T0569100-01'],
      dtype='object', length=2058)
Phenotype columns: Index(['Entry_Treatment_Replicate', 'Entry_Treatment',
       'Days till flowering (50%) (DTF)', 'Chlorophyll content index',
       'Leaf area index',
       'Flowering success (flowering and filled grain production occurred, yes or no)',
       'Fecundity', 'DTF_mean', 'DTF_std', 'DTF_cv', 'DTF_cv_flag', 'CCI_mean',
       'CCI_std', 'CCI_cv', 'CCI_cv_flag', 'LAI_mean', 'LAI_std', 'LAI_cv',
       'LAI_cv_flag', 'Flowering success_mean', 'Flowering success_

In [7]:
import pandas as pd
import numpy as np

# 读取原始文件
df = pd.read_csv(r"C:\Users\Lyttl\Desktop\rice\shuidao\data\combined_gene_phenotype_data_Fecundity.csv")

# 检查当前样本数量
n_original = df.shape[0]
n_target = 2000
n_needed = n_target - n_original

# 复制并添加微小扰动
df_extra = df.sample(n=n_needed, replace=True, random_state=42).copy()

# 对数值列添加高斯噪声
for col in df.columns:
    if df[col].dtype != 'object':  # 只对数值列操作
        std = df[col].std()
        if pd.notna(std) and std > 0:
            df_extra[col] += np.random.normal(0, std * 0.01, size=df_extra.shape[0])  # 添加1%标准差的噪声

# 合并原始和增强数据
df_augmented = pd.concat([df, df_extra], ignore_index=True)

# 重新编号样本ID（如果有“Sample”列）
if "Sample" in df_augmented.columns:
    df_augmented["Sample"] = [f"Sample_{i+1}" for i in range(df_augmented.shape[0])]

# 保存增强后的数据
df_augmented.to_csv(r"C:\Users\Lyttl\Desktop\rice\shuidao\data\expanded_combined_gene_phenotype_data_Fecundity.csv", index=False)

print(f"扩展完成：从 {n_original} 增加到 {df_augmented.shape[0]} 个样本")


扩展完成：从 1320 增加到 2000 个样本
